In [44]:

import pandas as pd
from sklearn.model_selection import train_test_split
from deepchem.data import NumpyDataset
from deepchem.data import DiskDataset
from deepchem.feat import MolGraphConvFeaturizer
from deepchem.feat.graph_data import GraphData
from numpy import array
from torch_geometric.data import Dataset, Data
import torch

In [45]:
train = pd.read_csv("../data/solubility-dataset-train.csv")
test = pd.read_csv("../data/solubility-dataset-test.csv")


In [46]:

# train, test_and_validation = train_test_split(dataset, test_size=0.2)
# validation, test = train_test_split(test_and_validation, test_size=0.5)

# train.to_csv("../data/train_s.csv")
# validation.to_csv("../data/validation_s.csv")
# test.to_csv("../data/test_s.csv")

train = NumpyDataset(train['SMILES'], y=train['logS'])
# validation = NumpyDataset(validation['SMILES'], y=validation['logS'])
test = NumpyDataset(test['SMILES'], y=test['logS'])

# transformer = MinMaxTransformer(transform_y=True, dataset=train)

# train = transformer.transform(train)
# validation = transformer.transform(validation)
# test = transformer.transform(test)

# with open("../data/scale_data_s.log", "w") as f:
#     f.write(f"{transformer.y_max},{transformer.y_min}")


def remove_pos_kwarg(mol: GraphData) -> GraphData:
    del mol.kwargs['pos']
    return mol


def featurize_dataset(dataset, featurizer) -> tuple:
    featurized = [featurizer.featurize(x)[0] for x in dataset.X]
    indices = [
        i for i, data in enumerate(featurized) if type(data) is GraphData
    ]
    return (
        array([featurized[i] for i in indices]),
        dataset.y[indices]
    )


featurizer = MolGraphConvFeaturizer(use_edges=True)

train_featurized, train_y = featurize_dataset(train, featurizer)
# validation_featurized, validation_y = featurize_dataset(validation, featurizer)
test_featurized, test_y = featurize_dataset(test, featurizer)

class SolubilityDataset(Dataset):
    
    def __init__(self, mols, sols):
        super(SolubilityDataset, self).__init__()
        self.data = []
        for mol, logs in zip(mols, sols):
            x = torch.tensor(mol.node_features).float()
            edge_index = torch.tensor(mol.edge_index)
            y = torch.tensor(logs).float()
            
            self.data.append(
                Data(x=x, edge_index=edge_index, y=y)
            )
    
    def len(self):
        return len(self.data)
    
    def get(self, idx):
        return self.data[idx]


test_torch_dataset = SolubilityDataset(test_featurized, test_y)
torch.save(test_torch_dataset, "test_torch_dataset.pt")

train_torch_dataset = SolubilityDataset(train_featurized, train_y)
torch.save(train_torch_dataset, "train_torch_dataset.pt")


# DiskDataset.from_numpy(
#     X=train_featurized,
#     y=train_y,
#     data_dir="../data/aqsoldb_train_s"
# )
# DiskDataset.from_numpy(
#     X=validation_featurized,
#     y=validation_y,
#     data_dir="../data/aqsoldb_valid_s"
# )
# DiskDataset.from_numpy(
#     X=test_featurized,
#     y=test_y,
#     data_dir="../data/aqsoldb_test_s"
# )


[01:50:58] Unusual charge on atom 0 number of radical electrons set to zero
[01:51:35] Explicit valence for atom # 19 O, 3, is greater than permitted
Failed to featurize datapoint 0, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
[01:52:00] Explicit valence for atom # 19 O, 3, is greater than permitted
Failed to featurize datapoint 0, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
[01:52:08] Explicit valence for atom # 19 O, 3, is greater than permitted
Failed to featurize datapoint 0, None. Appending empty array
Exception message: Python arg